In [ ]:
# -*- coding: utf-8 -*-

In [73]:
from __future__ import print_function
from __future__ import division
from prettytable import PrettyTable
import textwrap
# from tabulate import tabulate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import sys
import string
import subprocess
import cPickle
import editdistance
import nltk
import time
import math
from collections import Counter
import IPython
from IPython import display
from nltk.corpus import stopwords
from collections import defaultdict
from timeit import default_timer as timer
from matplotlib import rcParams
import networkx as nx
import dtw
import matplotlib.patches as patches
import scipy.io.wavfile
from python_speech_features import mfcc
from IPython.display import display
from matplotlib.ticker import MultipleLocator, \
     FormatStrFormatter, AutoMinorLocator
%matplotlib inline

In [67]:
def add_tick_lines(ax, words_list, axis_lim, horizontal=True):
    if horizontal:
        rot = 0
    else:
        rot = 90
    tick_lines = []
    for tup in words_list:
        start_val = tup.start / 100.0
        end_val = tup.end / 100.0
        text_val = tup.word.decode('utf-8')
        if text_val not in ['sil', 'sp']:
            ax.text((start_val + end_val)/2, \
                    axis_lim[1]+200, text_val, size=12, ha="center", va="center",\
                   rotation=rot)
        left, width = start_val, (end_val-start_val)
        bottom, height = axis_lim[1], 400
        right = left + width
        top = bottom + height
        # axes coordinates are 0,0 is bottom left and 1,1 is upper right
        p = patches.Rectangle((left, bottom), width, height,
            fill=False, clip_on=False)
        ax.add_patch(p)
        tick_lines.append(end_val)
    return ax, tick_lines
    

In [66]:
def plot_specgram_words(wav_fname, words_list):
    if not os.path.exists(wav_fname):
        print("wav file not found ...")

    fig = plt.figure()
    fig.set_size_inches(20, 3)
    ax = fig.add_subplot(111)
    sr, wav_data = scipy.io.wavfile.read(wav_fname)
    _ = plt.specgram(wav_data, Fs=sr, cmap=plt.cm.gist_heat)

    the_y_lim = ax.get_ylim()
    ax.grid(False)
    x_tick_lines = []
    ax, x_tick_lines = add_tick_lines(ax, words_list, the_y_lim, horizontal=True)
    ax.set_xticks(x_tick_lines, minor=True)
    ax.xaxis.grid(True, which="minor")
    for label in ax.get_yticklabels()[::2]:
        label.set_visible(False)
    ax.get_yticklabels()[-1].set_visible(True)


In [76]:
def plot_dtw(wav_1, wav_2):
    # Read wav data into numpy arrays
    sr1, y1 = scipy.io.wavfile.read(wav_1)
    mfcc1 = mfcc(y1, sr1)
    sr2, y2 = scipy.io.wavfile.read(wav_2)
    mfcc2 = mfcc(y2, sr2)
    print(mfcc1.shape, mfcc2.shape)
    wav_1_data = mfcc1
    wav_2_data = mfcc2
    
#     wav_1_data = np.fromfile(wav_1, dtype=np.float32)
#     wav_1_data = wav_1_data.reshape((-1,39))
#     wav_2_data = np.fromfile(wav_2, dtype=np.float32)
#     wav_2_data = wav_2_data.reshape((-1,39))
    
    # Compute dtw
    dist, cost, acc, path = dtw.dtw(wav_1_data, wav_2_data, \
                                    dist=lambda x, y: np.linalg.norm(x-y, ord=1))
    
    print("DTW distance: %f" %(dist))
    fig = plt.figure()
    fig.set_size_inches(20, 10)

    ax_0 = plt.subplot2grid((1,1),(0, 0))
    with sns.axes_style("white"):
        ax_0 = sns.heatmap(cost, cbar=False, xticklabels=100, yticklabels=100, ax=ax_0)
    ax_0.xaxis.tick_top()
    print(wav_1_data.shape, wav_2_data.shape)
    #return acc, cost

In [68]:
def local_dtw(A, B):
    dist, cost, acc, path = dtw.dtw(A, B, dist=lambda x, y: np.linalg.norm(x-y, ord=1))
    print("DTW distance: %f" %(dist))
    plt.figure(1)
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(12, 6)
    plt.subplot(121)
    meh_dtw.distance_cost_plot(cost)
    # Plotting accumulated cost
    plt.subplot(122)
    meh_dtw.distance_cost_plot(acc)
    plt.plot(path[0], path[1])

In [44]:
# test_wav = u'../../ZRTools/exp/callhome_train_20/matches/config0.80-0.90-0.80-50/wavs/n619.wav'
# test_words = [(0.1, .3, "haha"), (0.4, .7, "lolol")]
# plot_specgram_words(test_wav, test_words)

In [40]:
# !soxi $test_wav
# IPython.display.Audio(test_wav)


Input File     : '../../ZRTools/exp/callhome_train_20/matches/config0.80-0.90-0.80-50/wavs/n619.wav'
Channels       : 1
Sample Rate    : 8000
Precision      : 16-bit
Duration       : 00:00:00.83 = 6640 samples ~ 62.25 CDDA sectors
File Size      : 13.3k
Bit Rate       : 128k
Sample Encoding: 16-bit Signed Integer PCM

